# Localization Data Fetcher - Interactive UI

This notebook provides an interactive interface to fetch localization data from API and save to Excel while preserving validations.

In [33]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [34]:
# Import required libraries
import sys
import os
import json
import ipywidgets as widgets
from IPython.display import display, clear_output
import importlib

# Import functions from fetch_localization_preserve.py
import fetch_localization_preserve as flp

# Reload the module to get latest changes
importlib.reload(flp)
from unified_loader import UnifiedExcelReader, APIUploader, clean_nans

print("✓ All libraries imported successfully!")
print("✓ Functions loaded from fetch_localization_preserve.py")

✓ All libraries imported successfully!
✓ Functions loaded from fetch_localization_preserve.py


## Configuration

In [35]:
# Configuration
OUTPUT_FILE = "templates/localization.xlsx"
SHEET_NAME = "localization"
PARAMS = {
    "locale": "en_IN",
    "tenantId": "pb"
}
LANGUAGE_NAME = ""
TARGET_TENANT_ID = []

# Create UI widgets for input
base_url_input = widgets.Text(
    value='https://qa.digit.org',
    placeholder='Enter base URL',
    description='Base URL:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

tenant_id_input = widgets.Text(
    value='pb',
    placeholder='e.g., pb (State tenant)',
    description='State Tenant ID:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

target_tenant_id_input = widgets.Text(
    value='',
    placeholder='e.g., pb.citya, pb.cityb, pb.cityc (comma-separated for multiple)',
    description='Target Tenant IDs:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

language_input = widgets.Text(
    value='',
    placeholder='e.g., Hindi, Tamil, Punjabi (NEW language you want to add)',
    description='New Language:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

locale_code_input = widgets.Text(
    value='',
    placeholder='e.g., hi_IN, ta_IN, pa_IN (code for NEW language)',
    description='New Locale Code:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

output_label = widgets.HTML(value="")

fetch_button = widgets.Button(
    description='Fetch Data',
    button_style='success',
    tooltip='Click to fetch localization data',
    icon='download',
    layout=widgets.Layout(width='200px', height='40px')
)

print("✓ Configuration and UI widgets created!")

✓ Configuration and UI widgets created!


## Main Processing Function

In [42]:
def process_localization_data(button):
    """
    Main function to process localization data using functions from fetch_localization_preserve.py
    """
    global LANGUAGE_NAME, TARGET_TENANT_ID  # Make these accessible globally
    
    clear_output(wait=True)
    
    # Display the form again
    display(widgets.VBox([
        widgets.HTML(value="<h2>Localization Data Fetcher</h2>"),
        base_url_input,
        tenant_id_input,
        target_tenant_id_input,
        language_input,
        locale_code_input,
        fetch_button,
        output_label
    ]))
    
    # Get input values and update global variables
    base_url = base_url_input.value.strip()
    tenant_id = tenant_id_input.value.strip()
    locale_code = locale_code_input.value.strip()
    LANGUAGE_NAME = language_input.value.strip()
    
    # Parse comma-separated tenant IDs into a list
    target_tenant_input = target_tenant_id_input.value.strip()
    TARGET_TENANT_ID = [t.strip() for t in target_tenant_input.split(',') if t.strip()]
    
    # Validate inputs
    if not base_url:
        output_label.value = "<p style='color: red;'>❌ Error: Base URL is required</p>"
        return
    
    if not tenant_id:
        output_label.value = "<p style='color: red;'>❌ Error: Tenant ID is required</p>"
        return
    
    if not TARGET_TENANT_ID:
        output_label.value = "<p style='color: red;'>❌ Error: Target Tenant ID is required</p>"
        return
    
    if not locale_code:
        output_label.value = "<p style='color: red;'>❌ Error: Locale code is required</p>"
        return
        
    if not LANGUAGE_NAME:
        output_label.value = "<p style='color: red;'>❌ Error: Language name is required</p>"
        return
    
    # Display target tenants
    target_tenants_display = ', '.join(TARGET_TENANT_ID)
    
    output_label.value = f"""
    <div style='background-color: #f0f0f0; padding: 10px; border-radius: 5px;'>
        <h3>Configuration:</h3>
        <p>📍 Base URL: {base_url}</p>
        <p>🏢 State Tenant: {tenant_id}</p>
        <p>🎯 Target Tenants: {target_tenants_display} ({len(TARGET_TENANT_ID)} tenant(s))</p>
        <p>🌐 Locale Code: {locale_code}</p>
        <p>🗣️ Language Name: {LANGUAGE_NAME}</p>
        <hr>
        <p style='color: blue;'>⏳ Processing...</p>
    </div>
    """
    
    try:
        # Update PARAMS with user inputs
        PARAMS['locale'] = "en_IN"  # Keep locale fixed as en_IN for fetching all messages
        PARAMS['tenantId'] = tenant_id
        
        # Construct full URL
        full_url = f"{base_url}/localization/messages/v1/_search"
        
        print(f"[INFO] Working directory: {os.getcwd()}")
        print(f"[INFO] Full URL: {full_url}")
        print(f"[INFO] Params: {PARAMS}")
        print(f"[INFO] Language: {LANGUAGE_NAME} ({locale_code})")
        print(f"[INFO] Target Tenants: {', '.join(TARGET_TENANT_ID)}")
        
        # Step 1: Read existing translations
        print("\n[1/4] Reading existing translations...")
        existing_translations = flp.read_existing_translations(OUTPUT_FILE, SHEET_NAME)
        
        # Step 2: Fetch data from API
        print(f"\n[2/4] Fetching data from API for locale '{locale_code}'...")
        api_response = flp.fetch_localization_data(full_url, params=PARAMS)
        
        if not api_response:
            output_label.value = """<p style='color: red;'>❌ Failed to fetch data. Please check:</p>
            <ul>
                <li>Is the base URL correct?</li>
                <li>Is the API endpoint accessible?</li>
                <li>Do you need authentication/credentials?</li>
                <li>Check the console output for detailed error message</li>
            </ul>"""
            return
        
        # Step 3: Parse messages
        print("[3/4] Parsing messages...")
        messages = flp.parse_messages(api_response)
        
        if not messages:
            output_label.value = "<p style='color: red;'>❌ No messages found in response.</p>"
            return
        
        print(f"Found {len(messages)} messages")
        
        # Step 4: Create DataFrame and merge translations
        print(f"[4/4] Creating Excel structure with Locale = '{locale_code}'...")
        df = flp.create_dataframe(messages, locale_code)
        df = flp.merge_translations(df, existing_translations)
        
        # Save to Excel
        print(f"\nSaving to {OUTPUT_FILE} (preserving validations)...")
        flp.preserve_validations_and_save(df, OUTPUT_FILE, SHEET_NAME)
        
        # Verify file was saved
        if os.path.exists(OUTPUT_FILE):
            file_size = os.path.getsize(OUTPUT_FILE)
            file_size_kb = file_size / 1024
            print(f"✓ File saved successfully! Size: {file_size_kb:.2f} KB")
        
        # Get absolute path for download
        abs_path = os.path.abspath(OUTPUT_FILE)
        
        output_label.value = f"""<div style='background-color: #D4EDDA; padding: 15px; border-radius: 5px; border: 1px solid #C3E6CB;'>
            <h3 style='color: #155724;'>✅ Process Completed Successfully!</h3>
            <p>📁 File: {OUTPUT_FILE}</p>
            <p>📊 Total records: {len(df)}</p>
            <p>🌐 Locale: {locale_code}</p>
            <p>💾 File size: {file_size_kb:.2f} KB</p>
            <p>✓ All existing validations have been preserved</p>
            <p>✓ Existing translations have been merged</p>
            <hr>
            <p><strong>📥 Download:</strong> <a href="{OUTPUT_FILE}" download="{OUTPUT_FILE}" style="color: #0066CC; text-decoration: underline;">Click here to download {OUTPUT_FILE}</a></p>
        </div>"""
        
        # Display preview
        print("\n" + "="*60)
        print("Preview of first 5 records:")
        print("="*60)
        display(df.head())
        
        # Create download button widget
        from IPython.display import FileLink
        download_link = FileLink(OUTPUT_FILE, result_html_prefix="📥 Download file: ")
        display(download_link)
        
    except Exception as e:
        output_label.value = f"<p style='color: red;'>❌ Error: {str(e)}</p>"
        import traceback
        traceback.print_exc()

# Attach the event handler
fetch_button.on_click(process_localization_data)

print("✓ Main processing function defined and attached to button!")

✓ Main processing function defined and attached to button!


## Run the Application

Execute the cell below to display the interactive UI:

In [44]:
# Display the UI
display(widgets.VBox([
    widgets.HTML(value="<h2>Localization Data Fetcher</h2><p>Fill in the details below and click 'Fetch Data' to process:</p>"),
    base_url_input,
    tenant_id_input,
    target_tenant_id_input,
    language_input,
    locale_code_input,
    widgets.HTML(value="""
        <div style='background-color: #e7f3ff; padding: 10px; margin: 10px 0; border-left: 4px solid #2196F3; border-radius: 4px;'>
            <strong>ℹ️ Locale Code Format:</strong>
            <ul style='margin: 5px 0; padding-left: 20px;'>
                <li><strong>language</strong> = lowercase 2-letter code (en, hi, ta, bn, etc.)</li>
                <li><strong>COUNTRY</strong> = uppercase 2-letter code (IN, US, GB, etc.)</li>
            </ul>
            <strong>Examples:</strong> en_IN (English-India), hi_IN (Hindi-India), ta_IN (Tamil-India)
        </div>
    """),
    fetch_button,
    output_label
]))

---

## Upload & Validate Updated Localization File

After you have downloaded and updated the localization file with translations, use this section to upload and validate it before uploading to the system.

In [45]:
def validate_excel_file(file_content, filename, schema_file, display_name):
    """
    Validate Excel file content before saving
    Returns tuple: (is_valid, error_messages)
    """
    try:
        # Load ExcelValidator
        import sys
        sys.path.insert(0, os.path.abspath('.'))
        from mdms_validator import MDMSValidator as ExcelValidator
        
        # Create temp file for validation
        temp_dir = 'temp_validation'
        os.makedirs(temp_dir, exist_ok=True)
        temp_path = os.path.join(temp_dir, filename)
        
        # Write content to temp file
        with open(temp_path, 'wb') as f:
            f.write(file_content)
        
        # Validate
        validator = ExcelValidator(schemas_dir="schemas", templates_dir=temp_dir)
        result = validator.validate_file(filename, schema_file)
        
        # Clean up temp file
        os.remove(temp_path)
        
        if result['valid']:
            return True, []
        else:
            # Format error messages
            error_messages = []
            errors_by_sheet = {}
            
            for error in result['errors']:
                sheet = error.get('sheet', 'Unknown')
                if sheet not in errors_by_sheet:
                    errors_by_sheet[sheet] = []
                errors_by_sheet[sheet].append(error)
            
            for sheet_name, sheet_errors in errors_by_sheet.items():
                error_messages.append(f"\n   Sheet: {sheet_name}")
                for i, error in enumerate(sheet_errors[:3], 1):  # Show first 3 errors per sheet
                    if 'row' in error:
                        error_messages.append(
                            f"      {i}. Row {error['row']}, Column '{error['column']}'"
                        )
                        error_messages.append(f"         Error: {error['message']}")
                    else:
                        msg = error['message']
                        if msg.startswith(f"Sheet '{sheet_name}': "):
                            msg = msg.replace(f"Sheet '{sheet_name}': ", "")
                        error_messages.append(f"      {i}. {msg}")
                
                if len(sheet_errors) > 3:
                    error_messages.append(f"      ... and {len(sheet_errors) - 3} more errors")
            
            return False, error_messages
            
    except ImportError:
        # If validator not available, skip validation
        print(f"   ⚠️  Validator not available for {display_name}, saving without validation")
        return True, []
    except Exception as e:
        # If validation fails for any reason, show warning but allow save
        print(f"   ⚠️  Validation error for {display_name}: {str(e)}")
        return True, []

# Create file upload widget
localization_file_upload = widgets.FileUpload(
    accept='.xlsx,.xls',
    multiple=False,
    description='Select File:',
    layout=widgets.Layout(width='70%')
)

upload_validate_button = widgets.Button(
    description='⬆️ Validate & Upload File',
    button_style='primary',
    layout=widgets.Layout(width='95%', height='40px')
)

upload_output = widgets.Output()

def on_validate_upload(b):
    """Handler for validating and uploading the localization file"""
    global UPLOADED_LOCALIZATION_FILE
    
    with upload_output:
        clear_output()
        
        if not localization_file_upload.value:
            print("❌ No file selected. Please select a file first.")
            return
        
        print("="*70)
        print("  📋 VALIDATING UPLOADED FILE")
        print("="*70)
        print()
        
        uploaded_file = localization_file_upload.value[0]
        content = uploaded_file['content']
        filename = uploaded_file['name']
        
        print(f"[Localization File]")
        print(f"   📄 File: {filename}")
        print(f"   🔍 Validating...", end=" ")
        
        # Validate the file
        # Note: You need to specify the correct schema file for localization
        schema_file = 'localization_schema.yaml'
        display_name = 'Localization'
        
        is_valid, errors = validate_excel_file(content, filename, schema_file, display_name)
        
        if is_valid:
            print("✅ PASSED")
            # Save file to upload directory
            os.makedirs('upload', exist_ok=True)
            upload_path = os.path.join('upload', filename)
            with open(upload_path, 'wb') as f:
                f.write(content)
            
            UPLOADED_LOCALIZATION_FILE = upload_path
            
            print(f"   💾 Saved to: {upload_path}")
            print()
            print("="*70)
            print("  📊 VALIDATION SUMMARY")
            print("="*70)
            print(f"\n   ✅ File validated successfully!")
            print(f"\n   ➡️  File ready to be uploaded to the system.")
            print(f"   💾 File saved to: {upload_path}")
            print("="*70)
        else:
            print("❌ FAILED")
            print("   ⚠️  File NOT saved due to validation errors:")
            for error in errors:
                print(error)
            print()
            print("="*70)
            print("  📊 VALIDATION SUMMARY")
            print("="*70)
            print(f"\n   ❌ Validation failed")
            print("\n   💡 Fix the errors in your Excel file and try uploading again.")
            print("="*70)

upload_validate_button.on_click(on_validate_upload)

# Initialize global variable
UPLOADED_LOCALIZATION_FILE = None

# Display the upload UI
display(widgets.VBox([
    widgets.HTML(value="<h3>📁 Upload Updated Localization File</h3>"),
    widgets.HTML(value="<p><i>After filling in translations, upload your file here. It will be validated before being accepted.</i></p>"),
    localization_file_upload,
    widgets.HTML("<br>"),
    upload_validate_button,
    upload_output
]))

print("✓ File upload and validation UI created!")

✓ File upload and validation UI created!


In [39]:
reader = UnifiedExcelReader(UPLOADED_LOCALIZATION_FILE)
uploader = APIUploader()

---

## MODULE: LOCALIZATION 


**What it does:** Uploads language translations.

**Excel Sheet:** Localization

**Note:** Skip this if you only need English.

In [40]:
# Load Localization and Update Tenant Languages
print("="*60)
print("[MODULE: LOCALIZATION]")
print("="*60)

try:
    # Read localization data
    localization_data = reader.read_localization()
    
    if not localization_data:
        print("\n[INFO] No localization data found")
    else:
        print(f"\n[INFO] Loaded {len(localization_data)} translations")
        
        # Step 1: Upload Localization using dedicated localization API
        print("\n[1/2] Uploading localization messages...")
        result = uploader.create_localization_messages(
            localization_list=clean_nans(localization_data),
            tenant=PARAMS['tenantId']
        )
        
        if result['failed'] == 0:
            print("\n[SUCCESS] Localization messages uploaded successfully!")
        else:
            print(f"\n[WARNING] Some localizations failed. Check errors above.")
        
        # Step 2: Update tenant languages
        if TARGET_TENANT_ID and LANGUAGE_NAME:
            print(f"\n[2/2] Updating tenant languages for {len(TARGET_TENANT_ID)} tenant(s)...")
            
            # Get locale code from the localization data (first record's Locale field)
            locale_code = localization_data[0].get('locale', '') if localization_data else ''
            
            if locale_code and LANGUAGE_NAME:
                language_result = uploader.update_tenant_language(
                    tenant_ids=TARGET_TENANT_ID,
                    language_label=LANGUAGE_NAME,
                    language_value=locale_code,
                    state_tenant=PARAMS['tenantId']
                )
                
                if language_result['updated'] > 0:
                    print(f"\n[SUCCESS] Language added to {language_result['updated']} tenant(s)!")
                elif language_result['skipped'] > 0:
                    print(f"\n[INFO] Language already exists in all tenants (skipped {language_result['skipped']})")
                
                if language_result['failed'] > 0:
                    print(f"\n[WARNING] Failed to update {language_result['failed']} tenant(s). Check errors above.")
            else:
                print("\n[WARNING] Could not update tenant languages - locale code or language name missing")
        else:
            print("\n[INFO] Skipping tenant language update - no target tenants or language name specified")
            
except Exception as e:
    print(f"\n[ERROR] Localization process failed: {str(e)}")
    import traceback
    traceback.print_exc()

[MODULE: LOCALIZATION]

[INFO] Loaded 12752 translations

[1/2] Uploading localization messages...

[UPLOADING] Localization Messages
   Tenant: pg
   Total Messages: 12752
   API URL: http://localhost:8087/localization/messages/v1/_upsert

   Found 1 locales: en_IN
   [FAILED] Locale: en_IN
   ERROR: {"ResponseInfo":null,"Errors":[{"id":"ConstraintViolationException","parentId":null,"code":"An unhandled exception occurred on the server","message":"upsertMessages.messageRequest.messages[4230].message: must not be empty, upsertMessages.messageRequest.messages[4427].message: must not be empty, upsertMessages.messageRequest.messages[2127].message: must not be empty, upsertMessages.messageRequest.m
[SUMMARY] Created: 0
[SUMMARY] Already Exists: 0
[SUMMARY] Failed: 12752

[ERRORS] Found 1 error(s):
   - Locale: en_IN (12752 messages)
     Error: {"ResponseInfo":null,"Errors":[{"id":"ConstraintViolationException","parentId":null,"code":"An unhan

📊 ERROR REPORT UPDATED:
   File: errors/FAILE